# Binance Test Net API Research

In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import datetime

import requests
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import websocket
import json

import plotly.graph_objs as go

In [ ]:
### Testnet URLs
# https://testnet.binance.vision/api
# wss://testnet.binance.vision/ws-api/v3
# wss://testnet.binance.vision/ws
# wss://testnet.binance.vision/stream

# Binance Spot Test Network: https://testnet.binance.vision/
# API Key: odwF9bVsSsxjZnckgbSu3NfUgGqqJ2sow4OelwjEttIBB08r3Z5umQL0A03lp2Gd
# Secret key: Q3bcPKvbvlVpzv5BQe3lj7EkWdRhevEp24Oi7TENce6xO0FiXUNQKDa47QTyyKcK

In [ ]:
# Binance API endpoint for fetching historical kline data
url = 'https://testnet.binance.vision/api/v3/klines'
ws_url = 'wss://stream.binancefuture.com'

# Parameters for the API request
symbol = 'BTCUSDT'  # Bitcoin symbol
interval = '1h'    # 1-hour intervals
limit = 100        # Number of data points to fetch

# Send GET request to the Binance API
response = requests.get(url, params={'symbol': symbol, 'interval': interval, 'limit': limit})

# Parse the response JSON
data = response.json()

# Extract the closing prices from the data
prices = [float(entry[4]) for entry in data]

# Create a list of timestamps (optional)

timestamps = [entry[0] for entry in data]

# Create a time series plot
plt.plot(timestamps, prices)
plt.xlabel('Timestamp')
plt.ylabel('Price (USDT)')
plt.title('Bitcoin Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [4]:
# Global variables for storing price data
timestamps = []
prices = []
max_messages = 5  # Maximum number of messages to receive

def time_conv(time_stamp):
    return datetime.datetime.fromtimestamp(time_stamp/ 1000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-5]


def on_message(ws, message):
    data = json.loads(message)
    
    # Extract timestamp and price from the data
    timestamp = data['E']
    price = data['k'] #float(data['k']['c'])
    
    '''p_open = data['o']
    p_close = data['c']
    p_high = data['h']
    p_low = data['l']'''
    
    # Append timestamp and price to the lists
    timestamps.append(datetime.datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-5])
    #timestamps.append(timestamp) #uncomment for klines
    prices.append(price)

    # Terminate the WebSocket connection after reaching the maximum number of messages
    if len(timestamps) >= max_messages:
        ws.close()
    
    

# Binance WebSocket API endpoint for BTCUSDT symbol
ws_url = 'wss://testnet.binance.vision/ws/btcusdt@kline_1m'

# Start WebSocket connection
ws = websocket.WebSocketApp(ws_url, on_message=on_message)
ws.run_forever()


{'e': 'kline', 'E': 1685506317685, 's': 'BTCUSDT', 'k': {'t': 1685506260000, 'T': 1685506319999, 's': 'BTCUSDT', 'i': '1m', 'f': 3845916, 'L': 3845951, 'o': '27652.59000000', 'c': '27650.64000000', 'h': '27652.60000000', 'l': '27650.64000000', 'v': '0.59915900', 'n': 36, 'x': False, 'q': '16567.68244491', 'V': '0.01353700', 'Q': '374.33128620', 'B': '0'}}
{'e': 'kline', 'E': 1685506320177, 's': 'BTCUSDT', 'k': {'t': 1685506260000, 'T': 1685506319999, 's': 'BTCUSDT', 'i': '1m', 'f': 3845916, 'L': 3845952, 'o': '27652.59000000', 'c': '27650.64000000', 'h': '27652.60000000', 'l': '27650.64000000', 'v': '0.60015900', 'n': 37, 'x': True, 'q': '16595.33308491', 'V': '0.01453700', 'Q': '401.98192620', 'B': '0'}}
{'e': 'kline', 'E': 1685506322388, 's': 'BTCUSDT', 'k': {'t': 1685506320000, 'T': 1685506379999, 's': 'BTCUSDT', 'i': '1m', 'f': 3845953, 'L': 3845956, 'o': '27650.64000000', 'c': '27650.63000000', 'h': '27650.64000000', 'l': '27650.63000000', 'v': '0.00400000', 'n': 4, 'x': False, 'q

False

In [18]:
ws.close()

In [5]:
data

NameError: name 'data' is not defined

In [ ]:
timestamps

['2023-05-31 04:09:02.28900',
 '2023-05-31 04:09:05.10500',
 '2023-05-31 04:09:08.16500',
 '2023-05-31 04:09:11.47300',
 '2023-05-31 04:09:13.81400']